In [2]:
import pandas as pd
import os

# 📌 Cài đặt hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# 📂 Đường dẫn chứa các file
RAM_PATH = r"E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data"

# 📥 Hàm đọc và gắn tên hãng
def load_and_prepare(files, airline_name=None):
    dfs = []
    for file in files:
        full_path = os.path.join(RAM_PATH, file)
        print(f"📄 Đang đọc file: {full_path}")
        df = pd.read_csv(full_path)
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

        if "airline" not in df.columns and airline_name:
            df["airline"] = airline_name

        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# 🛫 Load từng hãng bay
df_vietjet = load_and_prepare(["vietjet_1.csv", "vietjet_2.csv"], airline_name="Vietjet Air")
df_vnairlines = load_and_prepare(["VNairlines_1_1.csv", "VNairlines_2_2.csv"], airline_name="Vietnam Airlines")
df_bamboo = load_and_prepare(["BambooAirway.csv"], airline_name="BAMBOO AIRWAYS")

# 🔗 Gộp toàn bộ dữ liệu
df_all = pd.concat([df_vietjet, df_vnairlines, df_bamboo], ignore_index=True)

# ✂️ Xoá các cột không cần thiết nếu có
cols_to_drop = ['source.name', 'departure_time', 'arrival_time', 'status']
df_all.drop(columns=[col for col in cols_to_drop if col in df_all.columns], inplace=True)

# 📆 Chuyển đổi cột ngày
df_all['flight_date'] = pd.to_datetime(df_all['date'], errors='coerce')

# ✈️ Tách flight_code và flight_number
def split_flight_id(fid):
    try:
        parts = fid.split("-")
        flight_code = parts[1]
        flight_number = ''.join(filter(str.isdigit, flight_code))
        return pd.Series([flight_code, flight_number])
    except:
        return pd.Series([None, None])

df_all[['flight_code', 'flight_number']] = df_all['flight_id'].apply(split_flight_id)

# 📆 Tách ngày/tháng/năm, ngày trong tuần
df_all['year'] = df_all['flight_date'].dt.year
df_all['month'] = df_all['flight_date'].dt.month
df_all['day'] = df_all['flight_date'].dt.day
df_all['dow'] = df_all['flight_date'].dt.dayofweek
df_all['is_weekend'] = df_all['dow'].isin([5, 6]).astype(int)

# ✅ Tạo các ID

# 1. Tạo flight_id mới (giữ lại flight_id gốc)
df_all.reset_index(inplace=True)
df_all.rename(columns={'index': 'flight_id', 'flight_id': 'original_flight_id', 'airline': 'airline_name'}, inplace=True)

# 2. Tạo price_id
df_all['price_id'] = range(len(df_all))

# 3. Tạo airline_id
dim_airline = pd.DataFrame(df_all['airline_name'].dropna().unique(), columns=['airline_name'])
dim_airline = dim_airline.sort_values(by='airline_name').reset_index(drop=True)
dim_airline['airline_id'] = dim_airline.index + 1
df_all = df_all.merge(dim_airline, on='airline_name', how='left')

# 4. Tạo fare_class_id
dim_fare_class = pd.DataFrame(df_all['fare_class'].dropna().unique(), columns=['fare_class'])
dim_fare_class = dim_fare_class.sort_values(by='fare_class').reset_index(drop=True)
dim_fare_class['fare_class_id'] = dim_fare_class.index + 1
df_all = df_all.merge(dim_fare_class, on='fare_class', how='left')

# 5. Tạo date_id
dim_date = pd.DataFrame(df_all['flight_date'].dropna().unique(), columns=['flight_date'])
dim_date = dim_date.sort_values(by='flight_date').reset_index(drop=True)
dim_date['date_id'] = dim_date.index + 1
df_all = df_all.merge(dim_date, on='flight_date', how='left')

# 6. Gán origin_airport_id và dest_airport_id

# Tạo bản đồ sân bay → thành phố
airport_city_map = {
    "SGN": "Ho Chi Minh City", "HAN": "Hanoi", "DAD": "Da Nang", "CXR": "Nha Trang",
    "HUI": "Hue", "DLI": "Da Lat", "PQC": "Phu Quoc", "VII": "Vinh", "VCA": "Can Tho",
    "PXU": "Pleiku", "BMV": "Buon Ma Thuot", "THD": "Thanh Hoa", "VDH": "Dong Hoi",
    "VCL": "Chu Lai", "UIH": "Quy Nhon", "DIN": "Dien Bien", "VKG": "Rach Gia",
    "CAH": "Ca Mau", "VCS": "Con Dao", "BMQ": "Phan Thiet", "CJM": "Con Dao - alt",
    "GLH": "Gia Lai", "NHA": "Nha Trang (alt)", "HAG": "Ha Giang", "YBM": "Yen Bai",
    "HPH": "Hai Phong"
}

# Tạo city từ origin (bản gốc)
df_all['city'] = df_all['origin'].map(airport_city_map)

# Tạo bảng sân bay duy nhất
all_airports = pd.concat([
    df_all[['origin', 'city']].rename(columns={'origin': 'airport_code'}),
    df_all[['destination']].rename(columns={'destination': 'airport_code'}).assign(city=None)
], ignore_index=True).drop_duplicates(subset='airport_code')

dim_airport = all_airports.drop_duplicates(subset='airport_code').sort_values('airport_code').reset_index(drop=True)
dim_airport['airport_id'] = dim_airport.index + 1

# Gán origin_airport_id
df_all = df_all.merge(dim_airport.rename(columns={'airport_code': 'origin'}), on='origin', how='left')
df_all.rename(columns={'airport_id': 'origin_airport_id'}, inplace=True)

# Gán dest_airport_id
df_all = df_all.merge(dim_airport.rename(columns={'airport_code': 'destination'}), on='destination', how='left')
df_all.rename(columns={'airport_id': 'dest_airport_id'}, inplace=True)

# 🧹 Dọn dẹp cột trung gian thừa
df_all.drop(columns=['city_x', 'city_y', 'origin_city', 'destination_city'], errors='ignore', inplace=True)

# 💾 Xuất file CSV hoàn chỉnh
final_output_path = r"E:\DATA-PROCESSING\Du_an_tot_nghiep\airline_cleaned3_fulldata.csv"
df_all.to_csv(final_output_path, index=False, encoding="utf-8-sig")
print(f"✅ Đã lưu file hoàn chỉnh: {final_output_path}")


📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\vietjet_1.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\vietjet_2.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\VNairlines_1_1.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\VNairlines_2_2.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\BambooAirway.csv
✅ Đã lưu file hoàn chỉnh: E:\DATA-PROCESSING\Du_an_tot_nghiep\airline_cleaned3_fulldata.csv
